In [8]:
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Load environment variables from .env file
load_dotenv()

# Get Spotify API credentials from environment variables
SPOTIPY_CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')
SPOTIPY_REDIRECT_URI = os.getenv('SPOTIPY_REDIRECT_URI')
SAIF_SONGS_ID = os.getenv('SAIF_SONGS_ID')

# All available Spotify scopes
SCOPES = [
    'ugc-image-upload', 'user-read-playback-state', 'user-modify-playback-state', 
    'user-read-currently-playing', 'app-remote-control', 'streaming', 
    'playlist-read-private', 'playlist-read-collaborative', 'playlist-modify-private', 
    'playlist-modify-public', 'user-follow-modify', 'user-follow-read', 
    'user-read-playback-position', 'user-top-read', 'user-read-recently-played', 
    'user-library-modify', 'user-library-read', 'user-read-email', 'user-read-private'
]

# Join all scopes into a space-separated string
SCOPE = ' '.join(SCOPES)

# Set up SpotifyOAuth
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,
                                               client_secret=SPOTIPY_CLIENT_SECRET,
                                               redirect_uri=SPOTIPY_REDIRECT_URI,
                                               scope=SCOPE))

# Main execution
playlist_name = "Saif Songs"
results = sp.playlist_tracks(SAIF_SONGS_ID)
tracks = results['items']
while results['next']:
    results = sp.next(results)
    tracks.extend(results['items'])

# Extract song and artist information
songs = []
for track in tracks:
    track_info = track['track']
    song_name = track_info['name']
    artists = ', '.join([artist['name'] for artist in track_info['artists']])
    songs.append(f"{song_name} by {artists}")

print(f"Songs in the playlist '{playlist_name}':")
for i, song in enumerate(songs, 1):
    print(f"{i}. {song}")

Songs in the playlist 'Saif Songs':
1. Layla by Cairokee
2. Atgah Wahed by Angham
3. Hateer Min Tany by Hamza Namira
4. Mine d'or by KALY
5. Origine by KALY
6. Menteuse by KALY, Dadju
7. Doucement by Ronisia
8. Problème by Ronisia, CKay
9. Désolée by Ronisia
10. Solitude by Ronisia
11. Copines by Aya Nakamura
12. Comportement by Aya Nakamura
13. Plus Jamais (feat. Stormzy) by Aya Nakamura, Stormzy
14. J'ai mal by Aya Nakamura
15. Haut niveau by Aya Nakamura
16. Definitivamente by Daddy Yankee, Sech
17. I Don't Want You Back by AJ Mitchell
18. El Campeón by El Kimiko y Yordy, Michel Boutic, EL YORDY DK
19. I Want It That Way by Backstreet Boys
20. It's Gonna Be Me by *NSYNC
21. Tearin' up My Heart - Radio Edit by *NSYNC
22. Bye Bye Bye - From Deadpool and Wolverine Soundtrack by *NSYNC
23. Leave Some Day by Kevo Muney
24. J'avoue, j'avoue by SenSey', Hiro
25. Django by Dadju, Franglish
26. Donna Imma by Franglish
27. Peur d'aimer by Franglish
28. Sans moi by Franglish, Aya Nakamura
29. 